In [1]:
import numpy as np
import rospy
import sys
#from geometry_msgs.msg import Twist
#from sensor_msgs.msg import LaserScan
#from std_srvs.srv import Empty
#from nav_msgs.msg import Odometry
#from sensor_msgs.msg import Imu
import time
from tqdm import tqdm
import random
import keyboard
#import Mobile_Control

from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan
from std_srvs.srv import Empty
from nav_msgs.msg import Odometry
from sensor_msgs.msg import Imu

In [2]:
cmd_vel="/cmd_vel"
laser_topic="/scan"
odom_topic="/odom"
imu_topic = "/imu"

In [3]:
class Mobile_Control:
    def __init__(self):
        self.cmd_vel = rospy.Publisher(cmd_vel,Twist,queue_size =1)
        self.subscriber = rospy.Subscriber(laser_topic,LaserScan,callback=self.laser_callback)
        self.subscriber_odo=rospy.Subscriber(odom_topic,Odometry,callback=self.odom_callback)
        self.subscriber_imu=rospy.Subscriber(imu_topic, Imu, callback=self.imu_callback)
        self.reset = rospy.ServiceProxy("/gazebo/reset_simulation",Empty)
        self.pause = rospy.ServiceProxy("/gazebo/pause_physics",Empty)
        self.unpause = rospy.ServiceProxy("/gazebo/unpause_physics",Empty)
        self.i =0
        self.count =0
        self.vel=Twist()
        self.vel.linear.x = 0
        self.vel.linear.y = 0
        self.vel.linear.z = 0
        self.vel.angular.x = 0
        self.vel.angular.y = 0
        self.vel.angular.z = 0
        
    def odom_callback(self,msg1):
        self.odom_data=msg1.twist.twist
        self.odom_pose=msg1.pose.pose.position
    def laser_callback(self, msg):
        self.laser_data=msg.ranges
        self.data=np.array(self.laser_data)
        self.laser_mask=np.logical_or(self.data>2, self.data<0)
        self.range_angels=np.arange(len(self.laser_data))
        self.i=0
        self.range_angels2=self.range_angels[self.laser_mask]
        for self.i in (self.range_angels2):
            self.data[self.i]=2
        self.data_laser=np.round(self.data[:])
    def imu_callback(self, msg_imu):
        self.imu_x=msg_imu.orientation.x
        self.imu_y=msg_imu.orientation.y

In [4]:
rospy.init_node('Mobile_Control',anonymous=True)

In [5]:
Robot = Mobile_Control()
Robot.reset()

In [6]:
#Hyperparameters
gamma = 0.9 
lr =0.6
lr_decay=0.999
num_episodes =10000
epsilon =0.4
epsilon_decay =0.999999

actions=[[1,0],[0.5,1],[0.5,-1],[0,2],[0,-2],[-0.8,0]]
legal_actions=len(actions)
s=np.zeros([3,3,3,3,3,3,3])   
#Q=np.zeros((3,3,3,3,3,3
Q=np.load('Q-table-sp.npy')
#Q=np.zeros((s.size,legal_actions))

In [3]:
import numpy as np
Q=np.load('Q-table-sp.npy')
s=[0,1,2,1,0,2,1]
s0=(int)(s[0]*81+s[1]*27+s[2]*9+s[3]*3+s[4]+243*s[5]+729*s[6])

for q in range(Q.shape[0]):
    print(Q[q])

[0.89224498 1.35287106 0.65788222 1.33280824 1.23355121 1.29330348]
[0.68686226 0.44633673 0.94713569 0.86517317 0.6888379  1.22773095]
[1.77460981 1.68545683 1.75263131 1.70754722 1.80033022 1.6386572 ]
[0.3589428 0.        0.        0.        0.        0.       ]
[0.84716688 0.43674932 0.         0.         0.         0.        ]
[0.6120259  0.43306419 1.55801384 0.84319037 2.16001384 0.3505569 ]
[0.4560268 0.        0.        0.        0.        0.       ]
[0.89964513 2.15217429 1.19009636 0.21128213 0.         0.75271177]
[1.90128941 1.76515889 1.97296123 1.9068924  1.99475484 1.76814217]
[0. 0. 0. 0. 0. 0.]
[0.         0.         0.         0.32450795 0.         0.        ]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0.1971171 0.        0.        0.        0.        0.       ]
[0.         0.         0.         0.         0.57064872 0.76023925]
[0. 0. 0. 0. 0. 0.]
[1.25268935 0.         0.         0.         0.         0.90246915]
[0.73805623 0.         3.09934076 1.6731337  1.1249844

[0.         1.35236557 0.         0.         0.         0.        ]
[0.63110814 0.         0.         0.         0.         0.        ]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[1.67294993 0.         0.         0.         0.         0.        ]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0.         0.         0.         0.         1.54594292 0.        ]
[1.21079481 1.4841276  1.60133175 1.60595284 1.25726676 1.17228172]
[1.64510326 0.         0.         0.         0.         0.        ]
[2.88266994 0.         0.85849486 2.00283055 1.22236577 0.        ]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0.94965604 0.         0.         0.         0.         0.        ]
[9.2631593 0.        0.        0.        0.        0.       ]
[1.03633993 0.         0.         0.         0.         0.        ]
[1.54741813 0.         0.         0.         0.         0.        ]
[3.74322786 0.    

[0.         2.13712278 0.         0.         0.         0.        ]
[3.72689299 3.87074713 5.54053393 3.91662514 2.93947008 2.93254576]
[5.71698941 4.89880211 4.86940705 4.66540096 4.88721246 4.89421742]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0.         0.         0.         0.         0.10206448 0.        ]
[1.77517156 0.         0.         0.         0.         0.        ]
[0.76034167 0.         2.39251203 0.         0.         0.        ]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[1.87381778 0.         0.         0.         0.         0.        ]
[1.0231157  2.64846059 1.78426284 4.03437714 2.52138395 2.19019712]
[0.75610718 0.         4.34783827 1.27825183 1.37136501 1.25937909]
[0.8496582 0.        0.        0.        0.        0.       ]
[1.83369795 3.65836019 0.         0.99404858 0.         1.81397842]
[4.46798691 1.56850745 5.86051359 1.81489121 0.         2.94089014]
[2.3142237

[5.35909514 4.64846801 4.70720304 4.76087261 5.3010277  5.19358155]
[2.03344683 2.09186116 1.86175747 3.10085334 1.89505606 1.89781126]
[3.05312085 3.01727683 2.89556191 2.66155124 2.45110664 3.00706734]
[2.87292061 3.21487884 3.40346126 3.15244423 2.95333887 3.47716307]
[0.89353234 0.50794033 1.86996957 0.7001458  0.         0.57899099]
[3.16161846 1.94011876 2.69505742 2.21377643 3.42233915 0.99876911]
[3.65792614 4.16170403 4.19717788 4.10914629 3.94607448 3.96567581]
[0. 0. 0. 0. 0. 0.]
[4.17967646 2.33924865 1.83361823 1.5027202  0.         0.        ]
[4.07248124 4.2027353  1.59553747 3.83736842 3.63178558 3.67105478]
[4.06262578 3.22025582 3.07692996 2.88534141 2.99141376 2.98827296]
[4.12533348 0.83505412 0.99562887 2.66844095 1.83937521 0.96920675]
[2.39062717 0.63822426 0.         0.         0.         0.        ]
[2.50937462 1.82080734 3.89771435 0.         2.43172171 0.62584607]
[3.96339744 4.95276777 4.16306135 3.53775772 4.06928321 3.80713309]
[5.38096797 3.7900183  3.829

[3.60086851 4.75324332 4.15212636 0.         1.91362762 0.62320546]
[3.40937566 3.72521215 4.42046097 3.7064852  3.38465386 2.59012848]
[4.30194014 4.343996   4.4078985  4.44082292 4.23199291 5.10377028]
[5.09993588 5.28783507 4.70528    4.85489334 4.52112858 4.49508421]
[1.92066248 1.34271437 1.28382918 2.84408685 1.56500627 1.94405284]
[0.65833274 1.11090038 0.         0.         1.20027882 0.00932045]
[1.71713565 0.         2.06857446 1.36377057 0.         1.02998189]
[0.         0.         0.         0.         0.         0.03832391]
[0.         0.         0.         0.03751527 0.         2.66973092]
[0.        0.        2.6759599 0.        0.        0.       ]
[0. 0. 0. 0. 0. 0.]
[1.05566983 0.         0.         0.         0.         0.41219011]
[0.         1.40525887 0.         0.         0.54750989 0.        ]
[2.6537652  2.63472236 2.79023479 2.93465978 2.677967   2.04360587]
[0.52646038 0.88581556 2.63920621 0.96052691 3.29968915 1.33242387]
[1.15077041 1.12654388 2.12561503 

[4.95382356 3.71207819 4.51601092 3.62323033 3.91917962 4.49655495]
[2.01902757 2.20978099 2.02157711 2.21499898 2.84477144 2.33569363]
[0.82210573 0.         0.         0.         0.         0.        ]
[0. 0. 0. 0. 0. 0.]
[4.16351635 1.51247706 3.11208648 0.         2.89665388 1.73996889]
[5.00600465 2.76781382 2.6338597  3.59588549 3.49349291 3.63291634]
[1.6081142  2.80049615 3.03169213 0.         5.11118741 0.        ]
[4.83191447 3.80596721 2.52821025 1.6223941  0.8676431  0.86165494]
[5.16346943 4.53050632 4.60935466 5.38033678 4.61105884 4.7198734 ]
[5.36416895 4.66154454 5.7870828  4.55857513 5.01432382 4.22990915]
[1.1267477  1.26440522 1.47381644 1.92940197 1.62336727 1.52972664]
[2.56223258 2.67555474 1.30948794 2.17689182 2.35169943 1.49583038]
[2.34701354 2.95073773 2.60659371 2.59102863 2.52815267 1.70021731]
[0. 0. 0. 0. 0. 0.]
[0.01635569 0.         0.         0.         0.         0.        ]
[1.98516019 0.         0.         1.30298112 1.32303765 0.10462573]
[0.02775

In [9]:
def get_state():
    sp_ang=Robot.odom_data.angular.z
    if(sp_ang<-0.2):
        ang_sp=0
    elif(sp_ang>0.2):
        ang_sp=1
    else:
        ang_sp=2    
    sp_lin=Robot.odom_data.linear.x
    if(sp_lin<-0.2):
        lin_sp=0
    elif(sp_lin>0.2):
        lin_sp=1
    else:
        lin_sp=2        
    s=Robot.data_laser
    s=np.append(s,[ang_sp,lin_sp])

       #state index
       #Robot.pause()
    s0=(int)(s[0]*81+s[1]*27+s[2]*9+s[3]*3+s[4]+243*s[5]+729*s[6])
    #epsilon greedy. to choose random actions initially when Q is all zeros
    return s,s0

In [10]:
print(Robot.imu_x)

0.00015510780301308209


In [19]:
while(1):
    (a,_)=get_state()

    print(a)
    time.sleep(1)

[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[2. 0. 0. 0. 2. 2. 0.]
[2. 1. 0. 1. 2. 2. 0.]
[2. 1. 1. 1. 2. 2. 2.]
[2. 0. 0. 1. 2. 2. 1.]
[1. 0. 0. 0. 2. 2. 1.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 2. 2.]
[0. 0. 0. 0. 2. 1. 2.]
[0. 0. 2. 2. 2. 1. 2.]
[0. 2. 2. 2. 2. 1. 2.]
[0. 0. 2. 2. 2. 0. 2.]
[0. 0. 0. 0. 2. 0. 2.]
[2. 0. 0. 0. 0. 0. 2.]
[2. 2. 2. 0. 0. 0. 2.]
[2. 2. 2. 0. 0. 2. 2.]
[2. 2. 2. 0. 0. 2. 2.]
[2. 2. 2. 0. 0. 2. 2.]
[2. 2. 2. 0. 0. 2. 2.]
[2. 2. 2. 0. 0. 2. 2.]
[2. 2. 2. 0. 0. 2. 2.]
[2. 2. 2. 0. 0. 2. 2.]
[2. 2. 2. 0

KeyboardInterrupt: 

In [10]:
# Train model
process_bar = tqdm(range(num_episodes))
for i in process_bar:

    if(abs(Robot.imu_x)>0.3 or abs(Robot.imu_y)>0.3):
        Robot.reset()
    if(abs(Robot.odom_pose.x)>8 or abs(Robot.odom_pose.y)>8):
        Robot.reset()

    s0=get_state()
    print('s0:',s0)
    #epsilon greedy. to choose random actions initially when Q is all zeros
    if np.random.random()<epsilon:
        action_index=np.random.randint(0,legal_actions)
        epsilon = epsilon*epsilon_decay
        #print('Took random action')
    else:
        action_index=np.argmax(Q[s0,:])

    a =actions[action_index]
    Robot.vel.linear.x = a[0]
    Robot.vel.angular.z=a[1]
    Robot.cmd_vel.publish(Robot.vel)

    time.sleep(0.1)
    #get next state index
    j=get_state()
    reward=max(Robot.odom_data.linear.x,0)

    Q[s0,action_index]= (1-lr)*Q[s0,action_index]+ lr*(reward+gamma*np.max(Q[j,:]))


  0%|          | 1/10000 [00:00<17:22,  9.60it/s]

s0: 2182
s0: 2182


  0%|          | 3/10000 [00:00<17:21,  9.60it/s]

s0: 1453
s0: 1210


  0%|          | 5/10000 [00:00<17:10,  9.70it/s]

...

100%|█████████▉| 9997/10000 [17:10<00:00,  9.73it/s]

s0: 1376
s0: 1376


100%|█████████▉| 9999/10000 [17:10<00:00,  9.75it/s]

s0: 1376
s0: 1376


100%|██████████| 10000/10000 [17:10<00:00,  9.70it/s]


In [11]:

#Test model
Q=np.load('Q-table-sp.npy')

while(1):
    if(abs(Robot.imu_x)>0.3 or abs(Robot.imu_y)>0.3):
        Robot.reset()
    if(abs(Robot.odom_pose.x)>8 or abs(Robot.odom_pose.x)>8):
        Robot.reset() 
    s=Robot.data_laser
    sp_ang=Robot.odom_data.angular.z
    if(sp_ang<-0.2):
        ang_sp=0
    elif(sp_ang>0.2):
        ang_sp=1
    else:
        ang_sp=2    
    sp_lin=Robot.odom_data.linear.x
    if(sp_lin<-0.2):
        lin_sp=0
    elif(sp_lin>0.2):
        lin_sp=1
    else:
        lin_sp=2

    s=np.append(s,[ang_sp,lin_sp])
    s0=(int)(s[0]*81+s[1]*27+s[2]*9+s[3]*3+s[4]+243*s[5]+729*s[6])
    print(s)
    #print('sp:',ang_sp,lin_sp)
    #print('s0:',s0)

    if np.random.random()<0.2:
        print("Random")
        action_index=np.random.randint(0,legal_actions)
        #a =actions[action_index]
    else:
        action_index=np.argmax(Q[s0,:])
        print('action index:',action_index)
        print(Q[s0,:])


    a =actions[action_index]
    Robot.vel.linear.x = a[0]
    Robot.vel.angular.z=a[1]
    Robot.cmd_vel.publish(Robot.vel)    
    print('-------------------')
    time.sleep(1)    


[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
Random
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
Random
-------------------
[2. 2. 2. 0. 0. 0. 2.]
action index: 1
[2.96463984 3.13174416 2.77046092 2.73948309 2.17737449 2.78867119]
-------------------
[2. 2. 2. 2. 2. 0. 1.]
action index: 0
[4.62401603 4.1842293  4.10033946 4.08623477 3.78545033 3.88346292]
-------------------
[2. 2. 2. 2. 2. 2. 1.]
action index: 1
[4.78961287 4.81677587 4.76623415 4.19792275 4.73043699 4.61690719]
-------------------
[2. 2. 2. 2. 2. 0. 1.]
action index: 0
[4.62401603 4.1842293  4.10033946 4.08623477 3.78545033 3.88346292]
-------------------
[2. 2. 2. 2

[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
Random
-------------------
[2. 0. 0. 0. 0. 2. 2.]
Random
-------------------
[2. 0. 0. 0. 2. 2. 0.]
action index: 5
[1.29820871 1.34066173 1.32499692 1.23963398 1.34627861 1.39755287]
-------------------
[2. 2. 1. 2. 2. 2. 0.]
action index: 4
[2.49614886 2.1820668  2.26974395 2.23719332 2.53916489 2.22123366]
-------------------
[2. 2. 2. 2. 2. 1. 2.]
action index: 1
[3.20678437 3.49364895 3.1196587  3.13684436 3.07364291 3.05158945]
-------------------
[2. 1. 2. 2. 2. 0. 1.]
action index: 1
[4.35556866 4.52698957 4.16034464 4.00213034 4.29325872 4.42851572]
-------------------
[2. 2. 1. 2

[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
Random
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695

[0. 0. 0. 0. 1. 0. 2.]
action index: 5
[0.71802837 0.77512343 0.77534752 0.73455372 0.76812213 0.79137067]
-------------------
[0. 0. 0. 1. 2. 2. 0.]
action index: 3
[1.08451984 1.03698676 1.29916543 1.34875703 1.34116932 0.94680934]
-------------------
[2. 2. 0. 0. 0. 0. 2.]
action index: 2
[1.43850868 1.47719845 1.82554574 1.75649962 1.30069473 1.41604253]
-------------------
[2. 0. 0. 0. 0. 1. 2.]
action index: 4
[0.75397875 0.73632598 0.80293593 0.8142681  0.82252573 0.79966895]
-------------------
[0. 0. 0. 2. 2. 1. 2.]
Random
-------------------
[2. 0. 0. 0. 0. 0. 2.]
action index: 1
[0.87669115 0.95967189 0.82501009 0.89946759 0.87833016 0.89799278]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695

[0. 0. 0. 2. 2. 1. 1.]
action index: 1
[2.37050397 2.59174138 2.13333727 1.75512141 2.25736156 2.11328322]
-------------------
[0. 0. 0. 0. 2. 2. 2.]
action index: 3
[0.73782061 0.73210609 0.72598509 0.84193176 0.7186297  0.71895346]
-------------------
[2. 2. 0. 0. 0. 0. 2.]
action index: 2
[1.43850868 1.47719845 1.82554574 1.75649962 1.30069473 1.41604253]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
action index: 1
[0.6806492  0.72659695 0.6818979  0.67408317 0.6921389  0.672167  ]
-------------------
[2. 0. 0. 0. 0. 2. 2.]
Random
-------------------
[2. 2. 0. 0. 1. 2. 0.]
action index: 3
[1.20178805 1.00744762 1.47679249 1.58079886 1.51240224 1.48501934]
-------------------
[2. 2. 2. 2. 1. 0. 2.]
action index: 2
[3.29657047 3.09325019 3.36399857 3.03587464 3.08581167 3.02878428]
-------------------
[2. 2. 2. 1. 1. 1. 1.]
action index: 1
[4.2706108  4.57801977 4.22979185 3.96808397 3.75444659 3.88997495]
-------------------
[2. 1. 2. 2. 1. 0. 1.]
action index: 0
[4.30120325 4.16986629

KeyboardInterrupt: 